In [1]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xr
import os 
import pandas as pd  
from datetime import datetime

# 生成文件目录

In [5]:
start_year = 2011
end_year = 2021

folder_path = 'D:/data/SSS/2011-2021/'  # 文件夹路径
file_names = os.listdir(folder_path)  
len(file_names)  

132

In [6]:
file_new = [] 
for i in range(len(file_names)):
    SSS = xr.open_dataset(folder_path+file_names[i])
    file_new.append(SSS)

In [7]:
len(file_new)

132

In [8]:
data = xr.concat(file_new,dim='time')
data_path = r'D:/data/SSS/2011-2021_SSS_concat.nc' 
data.to_netcdf(data_path)

# 读取合并数据

In [9]:
nc = Dataset(data_path)
print(nc.variables.keys())

for var in nc.variables.keys():
    data = nc.variables[var][:].data
    print(var,data.shape)

dict_keys(['time', 'depth', 'lat', 'lon', 'sos', 'dos', 'sos_error', 'dos_error', 'sea_ice_fraction'])
time (132,)
depth (1,)
lat (720,)
lon (1440,)
sos (132, 1, 720, 1440)
dos (132, 1, 720, 1440)
sos_error (132, 1, 720, 1440)
dos_error (132, 1, 720, 1440)
sea_ice_fraction (132, 1, 720, 1440)


In [10]:
nc.variables['sos'][:].mask.shape 

(132, 1, 720, 1440)

# 记录数据

In [12]:
lat = []
lon = []
sos = []
time = [] 

In [27]:
sos = nc.variables['sos'][:].data
sos[sos == -32767] = np.nan

In [28]:
sos = np.squeeze(sos)
sos.shape 

(132, 720, 1440)

In [23]:
sos_mask = nc.variables['sos'][:].mask

In [25]:
sos_mask = np.squeeze(sos_mask)
sos_mask.shape 

(132, 720, 1440)

## 获取 lat lon time 

In [16]:
temp = Dataset(folder_path + file_names[0])
lat.append(temp.variables['lat'][:].data)
lon.append(temp.variables['lon'][:].data)

In [17]:
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [18]:
len(time)

132

# 保存数据

In [19]:
len(depth)

0

In [20]:
time[0]

0

In [29]:
# 只保存风速和风速异常
#创建新文件
#new_NC.close()
new_NC = Dataset("D:/data/SSS/2011-2021_SSS_correct.nc", 'w', format='NETCDF4')

new_NC.createDimension('time', len(time))
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2011-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('sos', 'f', ("time","lat","lon"))
new_NC.createVariable('sos_mask', 'f', ("time","lat","lon"))

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['sos'][:]=np.array(sos)
new_NC.variables['sos_mask'][:]=np.array(sos_mask)

#最后记得关闭文件
new_NC.close()